In [ ]:
!nvidia-smi

Wed Dec 21 03:52:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import json
import glob
import re
import pandas as pd
import random
import cv2
import shutil
import glob

#Setup

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 45150, done.
remote: Counting objects: 100% (281/281), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 45150 (delta 131), reused 192 (delta 80), pack-reused 44869
Receiving objects: 100% (45150/45150), 338.41 MiB | 26.73 MiB/s, done.
Resolving deltas: 100% (31829/31829), done.


In [ ]:
%cd /content/PaddleOCR

/content/PaddleOCR


In [ ]:
!python -m pip install paddlepaddle-gpu==0.0.0.post112 -f https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://www.paddlepaddle.org.cn/whl/linux/gpu/develop.html
     |████████████████████████████████| 585.2 MB 8.2 kB/s 
     |████████████████████████████████| 385 kB 15.3 MB/s 


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 619 kB 14.7 MB/s 
     |████████████████████████████████| 4.9 MB 58.7 MB/s 
     |████████████████████████████████| 2.2 MB 58.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 8.8 MB 70.4 MB/s 
     |████████████████████████████████| 204 kB 73.4 MB/s 
     |████████████████████████████████| 132 kB 73.3 MB/s 
     |████████████████████████████████| 399 kB 81.4 MB/s 
     |████████████████████████████████| 2.3 MB 68.7 MB/s 
  Created wheel for lanms-neo: filename=lanms_neo-1.0.2-cp38-cp38-linux_x86_64.whl size=110775 sha256=9d9358a0ea261b3acc03216b9802c17d1c34746c95e36d6abd99b20632088f22
  Stored in directory: /root/.cache/pip/wheels/e2/a4/df/d69223851b1eb3a4d1b961ae86a9c08bba24412cd3939c5850
  Created wheel for Polygon3: filenam

#Download data

In [ ]:
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
## Download training data
!gdown --id 1YUO3HG-ZdtCUhcAJR2XCBGv-X3VPgfV4
!unzip uaic2022_training_data_update.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/uaic2022_training_data/labels/im1937.json  
  inflating: dataset/uaic2022_training_data/labels/im2740.json  
  inflating: dataset/uaic2022_training_data/labels/im1464.json  
  inflating: dataset/uaic2022_training_data/labels/im3435.json  
  inflating: dataset/uaic2022_training_data/labels/im1818.json  
  inflating: dataset/uaic2022_training_data/labels/im1500.json  
  inflating: dataset/uaic2022_training_data/labels/im3551.json  
  inflating: dataset/uaic2022_training_data/labels/im3799.json  
  inflating: dataset/uaic2022_training_data/labels/im0592.json  
  inflating: dataset/uaic2022_training_data/labels/im1995.json  
  inflating: dataset/uaic2022_training_data/labels/im2686.json  
  inflating: dataset/uaic2022_training_data/labels/im3770.json  
  inflating: dataset/uaic2022_training_data/labels/im0907.json  
  inflating: dataset/uaic2022_training_data/labels/im1721.json  
  inflating: dataset/uaic2022_training_

In [ ]:
## Download Public Valid data
!gdown --id 1frwoaHR24ETHKf4aBQbce6wSoZO-Bi80
!unzip uaic2022_public_valid.zip -d dataset

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1frwoaHR24ETHKf4aBQbce6wSoZO-Bi80
To: /content/PaddleOCR/uaic2022_public_valid.zip
100% 156M/156M [00:00<00:00, 203MB/s]
Archive:  uaic2022_public_valid.zip
   creating: dataset/uaic2022_public_valid/
   creating: dataset/uaic2022_public_valid/images/
  inflating: dataset/uaic2022_public_valid/images/im0300.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0194.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0254.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0172.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0065.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0134.jpg  
  inflating: dataset/uaic2022_public_valid/images/im0278.jpg  
  inflating: dataset/uaic2022_public_val

#Train

## Train model detection


### Preprocess for train model detection

In [ ]:
def convert(src):
    f = open(src)
    label = json.load(f)
    for item in label:
      item['transcription'] = item['text']
      del item['text']
    label_str = json.dumps(label)
    return label_str

Convert to format PaddleOCR

In [ ]:
root_img = './dataset/uaic2022_training_data/images/'
root_label = glob.glob('./dataset/uaic2022_training_data/labels/*')
random.shuffle(root_label)
num_imgs = len(root_label)

train_ratio = 0.7
val_ratio = 0.3

train_idx_start = int(num_imgs*train_ratio)
val_idx_start = int(num_imgs*(train_ratio))

os.makedirs('./dataset/det/train')
os.makedirs('./dataset/det/val')

train_label = open("./dataset/det/train_label.txt","w")
val_label = open("./dataset/det/val_label.txt","w")

output = './dataset/det'

for idx, file in enumerate(root_label):
    data = convert(file)
    img_name = os.path.basename(file).split('.')[0] + '.jpg'

    if idx < val_idx_start:
      shutil.copy(os.path.join(root_img,img_name), os.path.join(output,'train',img_name))
      train_label.write( 'train/'+img_name+ '\t'+f'{data}' + '\n')
    else:
      shutil.copy(os.path.join(root_img,img_name), os.path.join(output,'val',img_name))
      val_label.write( 'val/'+img_name+ '\t'+f'{data}' + '\n')

Download pretrain

In [ ]:
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.1/en_det/det_r50_db%2B%2B_icdar15_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/det_r50_db++_icdar15_train.tar -C ./pretrain_models

### Training

In [ ]:
!python3 tools/train.py -c ./configs/det/det_r50_db++_icdar15.yml -o   \
         Global.pretrained_model=./pretrain_models/det_r50_db++_icdar15_train/best_accuracy  \
         Global.epoch_num=12  \
         Global.save_model_dir=/content/drive/MyDrive/uaic2022  \
         Global.save_epoch_step=2 \
        Train.dataset.data_dir=./dataset/det/  \
        Train.dataset.label_file_list=['./dataset/det/train_label.txt'] \
        Train.loader.batch_size_per_card=8  \
        Eval.dataset.data_dir=./dataset/det/  \
        Eval.dataset.label_file_list=['./dataset/det/val_label.txt']

##Train model recognition

### Preprocess for train model recognition

In [ ]:
def order_points(pts):
    if isinstance(pts, list):
        pts = np.asarray(pts, dtype='float32')
    rect = np.zeros((4, 2), dtype='float32')
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def perspective_transform(img, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))
    return warped

In [ ]:
def crop(train_label_path, output_path):
  train_label = open(train_label_path,'r')
  train_file_list = train_label.readlines()
  crop_label = open(output_path, 'w', encoding="utf-8")

  count=0
  for line in tqdm(train_file_list):
      file_name, labels = line.strip().split('\t')
      labels = json.loads(labels)
      img_path = os.path.join(root_dir, file_name)
      image = cv2.imread(img_path)
      id=0
      for label in labels:
        pts = label['points']
        transcription = label['transcription']

        try:
          pts = [[int(x[0]), int(x[1])] for x in pts]
        except:
          continue

        if transcription == "###" or transcription == '#':
          continue

        cropped_img = perspective_transform(image, pts)
        img_crop_name = os.path.basename(file_name).split('.')[0] + '_' + str(id) + '.jpg'

        cv2.imwrite(os.path.join(os.path.dirname(output_path)  ,'img', img_crop_name), cropped_img)
        crop_label.write(f"img/{img_crop_name}\t{transcription}\n")
        id += 1
        count += 1

  print('Total box:', count)



Crop text box

In [ ]:
root_dir = '/content/PaddleOCR/dataset/det'

outpath = './dataset/rec_data'

if (not os.path.exists(outpath + '/img')):
  os.makedirs(outpath + '/img')

output_train=os.path.join(outpath,'train_crop.txt')
crop('/content/PaddleOCR/dataset/det/train_label.txt', output_train)

output_val=os.path.join(outpath,'val_crop.txt')
crop('/content/PaddleOCR/dataset/det/val_label.txt', output_val)

In [ ]:
## Download pretrain
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/en/rec_r50_vd_srn_train.tar -P ./pretrain_models
!tar -xf ./pretrain_models/rec_r50_vd_srn_train.tar -C ./pretrain_models

In [ ]:
## get dict vietnamese
!wget https://raw.githubusercontent.com/hungcao0402/PaddleOCR-Vietnamese/master1/ppocr/utils/dict/vi_vietnam.txt

### Training

In [ ]:
!python3 tools/train.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                        -o Global.pretrained_model=./pretrain_models/rec_r50_vd_srn_train/best_accuracy  \
                            Global.character_dict_path=vi_vietnam.txt \
                            Global.epoch_num=6  \
                            Global.save_epoch_step=2 \
                            Train.dataset.name='SimpleDataSet'  \
                            Train.dataset.data_dir=/content/PaddleOCR/dataset/rec_data  \
                            Train.dataset.label_file_list=['./dataset/rec_data/train_crop.txt'] \
                            Eval.dataset.name='SimpleDataSet'  \
                            Eval.dataset.data_dir=/content/PaddleOCR/dataset/rec_data  \
                            Eval.dataset.label_file_list=['./dataset/rec_data/val_crop.txt']

#Evaluation

## Evaluate detection


In [ ]:
!python3 tools/eval.py -c ./configs/det/det_r50_db++_icdar15.yml  \
                       -o Global.pretrained_model=./output/det_r50_icdar15/latest.pdparams  \
                          Eval.dataset.data_dir=./dataset/det  \
                          Eval.dataset.label_file_list=['./dataset/det/val_label.txt']

[2022/12/13 07:44:36] ppocr INFO: Architecture : 
[2022/12/13 07:44:36] ppocr INFO:     Backbone : 
[2022/12/13 07:44:36] ppocr INFO:         dcn_stage : [False, True, True, True]
[2022/12/13 07:44:36] ppocr INFO:         layers : 50
[2022/12/13 07:44:36] ppocr INFO:         name : ResNet
[2022/12/13 07:44:36] ppocr INFO:     Head : 
[2022/12/13 07:44:36] ppocr INFO:         k : 50
[2022/12/13 07:44:36] ppocr INFO:         name : DBHead
[2022/12/13 07:44:36] ppocr INFO:     Neck : 
[2022/12/13 07:44:36] ppocr INFO:         name : DBFPN
[2022/12/13 07:44:36] ppocr INFO:         out_channels : 256
[2022/12/13 07:44:36] ppocr INFO:         use_asf : True
[2022/12/13 07:44:36] ppocr INFO:     Transform : None
[2022/12/13 07:44:36] ppocr INFO:     algorithm : DB++
[2022/12/13 07:44:36] ppocr INFO:     model_type : det
[2022/12/13 07:44:36] ppocr INFO: Eval : 
[2022/12/13 07:44:36] ppocr INFO:     dataset : 
[2022/12/13 07:44:36] ppocr INFO:         data_dir : ./dataset/det
[2022/12/13 07:44

## Evaluate recognition


In [ ]:
!python3 tools/eval.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                       -o Global.checkpoints=./output/rec/srn_new/latest \
                          Global.character_type=ch  \
                          Global.character_dict_path=./vi_vietnam.txt \
                          Eval.dataset.name='SimpleDataSet'  \
                          Eval.dataset.data_dir=./dataset/rec_data  \
                          Eval.dataset.label_file_list=['./dataset/rec_data/val_crop.txt']

[2022/12/13 08:34:55] ppocr INFO: Architecture : 
[2022/12/13 08:34:55] ppocr INFO:     Backbone : 
[2022/12/13 08:34:55] ppocr INFO:         name : ResNetFPN
[2022/12/13 08:34:55] ppocr INFO:     Head : 
[2022/12/13 08:34:55] ppocr INFO:         hidden_dims : 512
[2022/12/13 08:34:55] ppocr INFO:         max_text_length : 25
[2022/12/13 08:34:55] ppocr INFO:         name : SRNHead
[2022/12/13 08:34:55] ppocr INFO:         num_decoder_TUs : 4
[2022/12/13 08:34:55] ppocr INFO:         num_encoder_TUs : 2
[2022/12/13 08:34:55] ppocr INFO:         num_heads : 8
[2022/12/13 08:34:55] ppocr INFO:     Transform : None
[2022/12/13 08:34:55] ppocr INFO:     algorithm : SRN
[2022/12/13 08:34:55] ppocr INFO:     in_channels : 1
[2022/12/13 08:34:55] ppocr INFO:     model_type : rec
[2022/12/13 08:34:55] ppocr INFO: Eval : 
[2022/12/13 08:34:55] ppocr INFO:     dataset : 
[2022/12/13 08:34:55] ppocr INFO:         data_dir : ./dataset/rec_data
[2022/12/13 08:34:55] ppocr INFO:         label_file_l

#Convert to inference model

In [ ]:
## Export detection model
!python3 tools/export_model.py -c ./configs/det/det_r50_db++_icdar15.yml  \
                               -o Global.pretrained_model=./output/det_r50_icdar15/latest \
                                  Global.save_inference_dir=./inference/det

W1213 08:37:19.793715  1561 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 11.2
W1213 08:37:19.796355  1561 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022/12/13 08:37:21] ppocr INFO: load pretrain successful from ./output/det_r50_icdar15/latest
I1213 08:37:23.462684  1561 interpretercore.cc:239] New Executor is Running.
[2022/12/13 08:37:24] ppocr INFO: inference model is saved to ./inference/det/inference


In [ ]:
## Export recognition model
!python3 tools/export_model.py -c ./configs/rec/rec_r50_fpn_srn.yml  \
                               -o Global.pretrained_model=./output/rec/srn_new/latest \
                                  Global.character_dict_path=./vi_vietnam.txt \
                                  Global.save_inference_dir=./inference/rec

W1213 08:37:27.912516  1579 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 11.2, Runtime API Version: 11.2
W1213 08:37:27.914903  1579 gpu_resources.cc:91] device: 0, cuDNN Version: 8.1.
[2022/12/13 08:37:29] ppocr INFO: load pretrain successful from ./output/rec/srn_new/latest
I1213 08:37:33.767693  1579 interpretercore.cc:239] New Executor is Running.
[2022/12/13 08:37:36] ppocr INFO: inference model is saved to ./inference/rec/inference


Sau khi convert xong thì thử predict

In [ ]:
#predict detection
!python3 tools/infer/predict_det.py --use_gpu=True \
                                    --det_model_dir="./inference/det"  \
                                    --image_dir=./dataset/uaic2022_trainset_demo/images/im0012.jpg

[2022/12/13 08:37:43] ppocr INFO: im0012.jpg	[[[745.0, 226.0], [791.0, 226.0], [791.0, 253.0], [745.0, 253.0]], [[788.0, 225.0], [824.0, 225.0], [824.0, 253.0], [788.0, 253.0]], [[819.0, 222.0], [852.0, 222.0], [852.0, 253.0], [819.0, 253.0]], [[796.0, 199.0], [864.0, 195.0], [866.0, 225.0], [797.0, 228.0]], [[858.0, 195.0], [930.0, 189.0], [933.0, 219.0], [860.0, 225.0]], [[739.0, 148.0], [799.0, 132.0], [809.0, 171.0], [749.0, 186.0]], [[795.0, 132.0], [859.0, 120.0], [867.0, 162.0], [803.0, 174.0]], [[757.0, 104.0], [833.0, 86.0], [843.0, 126.0], [767.0, 144.0]], [[824.0, 85.0], [886.0, 70.0], [896.0, 114.0], [834.0, 128.0]], [[745.0, 76.0], [797.0, 59.0], [809.0, 96.0], [757.0, 113.0]], [[789.0, 59.0], [838.0, 42.0], [852.0, 81.0], [802.0, 98.0]]]

[2022/12/13 08:37:43] ppocr INFO: 0 The predict time of ./dataset/uaic2022_trainset_demo/images/im0012.jpg: 1.883923053741455
[2022/12/13 08:37:43] ppocr INFO: The visualized image saved in ./inference_results/det_res_im0012.jpg


In [ ]:
#predict recognition
!python3 tools/infer/predict_rec.py \
          --image_dir="./dataset/rec_data/img/im0001_0.jpg" \
          --rec_model_dir="./inference/rec/" --rec_image_shape="1,64,256" \
          --rec_algorithm="SRN"  \
          --rec_char_dict_path="./vi_vietnam.txt" --use_space_char=False

[2022/12/13 08:37:50] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2022/12/13 08:37:52] ppocr INFO: Predicts of ./dataset/rec_data/img/im0001_0.jpg:('NƠI', 0.9387781023979187)


#Text Detection and Recognition Inference Concatenation

In [ ]:
#get font to visualize
!gdown --id 1y5ijIAEOtHR20TCSKETO-9XHH-veCGID

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1y5ijIAEOtHR20TCSKETO-9XHH-veCGID
To: /content/PaddleOCR/font-times-new-roman.ttf
100% 611k/611k [00:00<00:00, 151MB/s]


In [ ]:
!python3 ./tools/infer/predict_system.py  \
                    --use_gpu=True  \
                    --det_model_dir="/content/drive/MyDrive/source/inference/SAST"  \
                    --rec_algorithm="SRN" \
                    --rec_model_dir="/content/drive/MyDrive/source/inference/SRN"  \
                    --rec_image_shape="1, 64, 256"  \
                    --drop_score=0.001  \
                    --image_dir="/content/PaddleOCR/dataset/det/val/" \
                    --rec_char_dict_path="./vi_vietnam.txt" \
                    --use_space_char=True  \
                    --vis_font_path=font-times-new-roman.ttf

[2022/12/19 14:14:55] ppocr INFO: In PP-OCRv3, rec_image_shape parameter defaults to '3, 48, 320', if you are using recognition model with PP-OCRv2 or an older version, please set --rec_image_shape='3,32,320
[2022/12/19 14:14:56] ppocr DEBUG: dt_boxes num : 0, elapse : 1.7786836624145508
[2022/12/19 14:14:56] ppocr DEBUG: rec_res num  : 0, elapse : 1.6689300537109375e-06
[2022/12/19 14:14:56] ppocr DEBUG: 0  Predict time of /content/PaddleOCR/dataset/det/val/im0048.jpg: 1.779s
[2022/12/19 14:14:57] ppocr DEBUG: The visualized image saved in ./inference_results/im0048.jpg
[2022/12/19 14:14:57] ppocr INFO: The predict total time is 1.790339469909668


# Evaluate E2E

In [ ]:
#convert to format paddle eval e2e
!python3 tools/end2end/convert_ppocr_label.py --mode=gt --label_path=./dataset/det/val_label.txt --save_folder=save_gt_label
!python3 tools/end2end/convert_ppocr_label.py --mode=pred --label_path=./inference_results/system_results.txt --save_folder=save_infer

The convert label saved in save_gt_label
The convert label saved in save_infer


In [ ]:
!python3 tools/end2end/eval_end2end.py ./save_gt_label/ ./save_infer/

start testing...
hit, dt_count, gt_count 1831 2879 15232
character_acc: 10.84%
avg_edit_dist_field: 3.62
avg_edit_dist_img: 41.25
precision: 63.60%
recall: 12.02%
fmeasure: 20.22%


# Test some image

In [ ]:
!python3 tools/end2end/convert_ppocr_label.py --mode=pred \
                                              --label_path=./inference_results/system_results.txt \
                                              --save_folder=save_infer

In [ ]:
root = glob.glob('./save_infer/*')
os.mkdir('result')
for file in root:
    file_name = os.path.basename(file).replace('.jpg','')
    f = open(file,'r')
    lines = f.readlines()

    output = open('result/'+file_name,'w')
    for line in lines:
        line=line.strip()
        label = line.split('\t')
        bbox = label[:8]
        trans = label[-1]
        data = ','.join(bbox)+ ',' +trans+'\n'
        output.write(data)